Todo:
- Figure out user authentication
    - See if there's different syntax for working with user playlists
    - Probably don't go too deep w/ user authentication without planning a UI bc it's going to be tested by others
- Use artist and album to figure out label

In [79]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from pprint import pprint
cid = ''
secret = ''
username = 'nshsunmalb8t3brxxz0ks3e47'
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=secret))

In [80]:
# Dictionary for mapping key signature integer value to pitch scale
int_to_key = {'-1': 'no key detected', '0': 'C', '1': 'C#', '2': 'D', '3': 'D#', '4': 'E',
              '5': 'E#', '6': 'F', '7': 'F#', '8': 'G', '9': 'G#', '10': 'A', '11': 'A#', '12': 'B'}

In [81]:
# Transpose numeric key + boolean minor/major to camelot wheel
# '01': '8B', '00': '5A', '11': '3B', '10:'

In [82]:
# Example playlist: "wow im at the beach"
playlist_uri = 'spotify:playlist:5CF6KvWn85N6DoWufOjP5T'

In [83]:
offset = 0
jibjob = []

while True:
    response = sp.playlist_items(playlist_uri,
                                offset=offset,
                                fields='items.track.id,items.track.artists,items.track.name,items.track.album,total',
                                additional_types=['track'])
    
    if len(response['items']) == 0:
        # Combine inner lists and exit loop
        # Todo: ask how this comprehension actually works
        jibjob = [j for i in jibjob for j in i]
        break
    
    jibjob.append(response['items'])
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

100 / 209
200 / 209
209 / 209


In [84]:
df = pd.DataFrame(columns=['artist', 'track_name', 'id', 'album'], index=range(len(jibjob)))


In [85]:
for i in range(len(jibjob)):
    df['artist'].iloc[i] = jibjob[i]['track']['artists'][0]['name']
    df['track_name'].iloc[i] = jibjob[i]['track']['name']
    df['id'].iloc[i] = jibjob[i]['track']['id']
    df['album'].iloc[i] = jibjob[i]['track']['album']['name']


In [86]:
df

,artist,track_name,id,album
0,Tabu Ley Rochereau,Laiss Toi Aime',4MTiPNTHdAefzgJ7qgp3rY,Laiss Toi Aime
1,Negro Leo,Tudo Foi Feito pra Gente Lacrar,3k02yoh9Sjhjd6rKef1jyN,Desejo de Lacrar
2,Shad,Rose Garden,61kT5zIV1Z9sPbdAlKRExr,TSOL
3,Tabu Ley Rochereau,Marie Maria,2i7hxvOTfTjtsEJuWYnKg0,Laiss Toi Aime
4,Tabu Ley Rochereau,Mokrano,0gflenw36L3PVpVUqtRZJd,Laiss Toi Aime
...,...,...,...,...
204,Mdou Moctar,Tala Tannam - Debmaster Remix,4W3iduYwzo3hbpvOnN1Otw,Tala Tannam (Debmaster Remix)
205,The Cyrkle,The Visit (She Was Here),5dIcC2RTD2BOHTC1F0jrIN,Red Rubber Ball (A Collection)
206,Gilberto Gil,Era nova,6wPo638YVloLLGDPZRBE2F,Refavela
207,Paco de Lucía,Entre Dos Aguas - Instrumental,3Fgk17TfdjXWFroJy29pXH,Fuente Y Caudal


In [87]:
# Create DF for audio features while avoiding rate limit
# Todo: split into 50-piece slices to reduce compute time and total queries

features = []
for i in range(len(jibjob)):
    features.append(sp.audio_features(df['id'][i]))


In [88]:
features = [j for i in features for j in i]

In [89]:
df_features = pd.DataFrame(features)

In [90]:
df_features


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.777,0.521,0,-11.669,1,0.0376,0.70100,0.585000,0.1780,0.852,132.733,audio_features,4MTiPNTHdAefzgJ7qgp3rY,spotify:track:4MTiPNTHdAefzgJ7qgp3rY,https://api.spotify.com/v1/tracks/4MTiPNTHdAef...,https://api.spotify.com/v1/audio-analysis/4MTi...,254520,4
1,0.573,0.408,8,-8.587,0,0.0373,0.92500,0.000147,0.0953,0.333,124.198,audio_features,3k02yoh9Sjhjd6rKef1jyN,spotify:track:3k02yoh9Sjhjd6rKef1jyN,https://api.spotify.com/v1/tracks/3k02yoh9Sjhj...,https://api.spotify.com/v1/audio-analysis/3k02...,235882,4
2,0.596,0.919,11,-4.451,1,0.3320,0.04930,0.000001,0.3050,0.835,183.331,audio_features,61kT5zIV1Z9sPbdAlKRExr,spotify:track:61kT5zIV1Z9sPbdAlKRExr,https://api.spotify.com/v1/tracks/61kT5zIV1Z9s...,https://api.spotify.com/v1/audio-analysis/61kT...,179200,4
3,0.550,0.341,10,-11.803,1,0.0414,0.73100,0.871000,0.1410,0.405,89.147,audio_features,2i7hxvOTfTjtsEJuWYnKg0,spotify:track:2i7hxvOTfTjtsEJuWYnKg0,https://api.spotify.com/v1/tracks/2i7hxvOTfTjt...,https://api.spotify.com/v1/audio-analysis/2i7h...,341040,4
4,0.696,0.582,9,-12.348,1,0.0410,0.55500,0.908000,0.1440,0.946,129.512,audio_features,0gflenw36L3PVpVUqtRZJd,spotify:track:0gflenw36L3PVpVUqtRZJd,https://api.spotify.com/v1/tracks/0gflenw36L3P...,https://api.spotify.com/v1/audio-analysis/0gfl...,326760,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.523,0.675,2,-8.344,1,0.0301,0.00617,0.017100,0.3720,0.196,84.038,audio_features,4W3iduYwzo3hbpvOnN1Otw,spotify:track:4W3iduYwzo3hbpvOnN1Otw,https://api.spotify.com/v1/tracks/4W3iduYwzo3h...,https://api.spotify.com/v1/audio-analysis/4W3i...,175714,4
205,0.696,0.133,7,-20.836,1,0.0387,0.71400,0.000010,0.1100,0.653,116.456,audio_features,5dIcC2RTD2BOHTC1F0jrIN,spotify:track:5dIcC2RTD2BOHTC1F0jrIN,https://api.spotify.com/v1/tracks/5dIcC2RTD2BO...,https://api.spotify.com/v1/audio-analysis/5dIc...,138133,4
206,0.528,0.446,7,-9.587,1,0.0360,0.77600,0.002530,0.0903,0.397,77.518,audio_features,6wPo638YVloLLGDPZRBE2F,spotify:track:6wPo638YVloLLGDPZRBE2F,https://api.spotify.com/v1/tracks/6wPo638YVloL...,https://api.spotify.com/v1/audio-analysis/6wPo...,238880,4
207,0.458,0.657,4,-12.020,0,0.0346,0.93300,0.878000,0.0842,0.914,102.450,audio_features,3Fgk17TfdjXWFroJy29pXH,spotify:track:3Fgk17TfdjXWFroJy29pXH,https://api.spotify.com/v1/tracks/3Fgk17TfdjXW...,https://api.spotify.com/v1/audio-analysis/3Fgk...,361867,4


In [91]:
merged = df.combine_first(df_features.drop(columns=['track_href', 'analysis_url', 'uri', 'type']))
merged

,acousticness,album,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,valence
0,0.70100,Laiss Toi Aime,Tabu Ley Rochereau,0.777,254520,0.521,4MTiPNTHdAefzgJ7qgp3rY,0.585000,0,0.1780,-11.669,1,0.0376,132.733,4,Laiss Toi Aime',0.852
1,0.92500,Desejo de Lacrar,Negro Leo,0.573,235882,0.408,3k02yoh9Sjhjd6rKef1jyN,0.000147,8,0.0953,-8.587,0,0.0373,124.198,4,Tudo Foi Feito pra Gente Lacrar,0.333
2,0.04930,TSOL,Shad,0.596,179200,0.919,61kT5zIV1Z9sPbdAlKRExr,0.000001,11,0.3050,-4.451,1,0.3320,183.331,4,Rose Garden,0.835
3,0.73100,Laiss Toi Aime,Tabu Ley Rochereau,0.550,341040,0.341,2i7hxvOTfTjtsEJuWYnKg0,0.871000,10,0.1410,-11.803,1,0.0414,89.147,4,Marie Maria,0.405
4,0.55500,Laiss Toi Aime,Tabu Ley Rochereau,0.696,326760,0.582,0gflenw36L3PVpVUqtRZJd,0.908000,9,0.1440,-12.348,1,0.0410,129.512,4,Mokrano,0.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.00617,Tala Tannam (Debmaster Remix),Mdou Moctar,0.523,175714,0.675,4W3iduYwzo3hbpvOnN1Otw,0.017100,2,0.3720,-8.344,1,0.0301,84.038,4,Tala Tannam - Debmaster Remix,0.196
205,0.71400,Red Rubber Ball (A Collection),The Cyrkle,0.696,138133,0.133,5dIcC2RTD2BOHTC1F0jrIN,0.000010,7,0.1100,-20.836,1,0.0387,116.456,4,The Visit (She Was Here),0.653
206,0.77600,Refavela,Gilberto Gil,0.528,238880,0.446,6wPo638YVloLLGDPZRBE2F,0.002530,7,0.0903,-9.587,1,0.0360,77.518,4,Era nova,0.397
207,0.93300,Fuente Y Caudal,Paco de Lucía,0.458,361867,0.657,3Fgk17TfdjXWFroJy29pXH,0.878000,4,0.0842,-12.020,0,0.0346,102.450,4,Entre Dos Aguas - Instrumental,0.914


In [92]:
for i in range(len(merged)):
    merged['key'].iloc[i] = int_to_key[str(merged['key'].iloc[i])]


/tmp/ipykernel_27795/1816759567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['key'].iloc[i] = int_to_key[str(merged['key'].iloc[i])]


In [93]:
merged

,acousticness,album,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,valence
0,0.70100,Laiss Toi Aime,Tabu Ley Rochereau,0.777,254520,0.521,4MTiPNTHdAefzgJ7qgp3rY,0.585000,C,0.1780,-11.669,1,0.0376,132.733,4,Laiss Toi Aime',0.852
1,0.92500,Desejo de Lacrar,Negro Leo,0.573,235882,0.408,3k02yoh9Sjhjd6rKef1jyN,0.000147,G,0.0953,-8.587,0,0.0373,124.198,4,Tudo Foi Feito pra Gente Lacrar,0.333
2,0.04930,TSOL,Shad,0.596,179200,0.919,61kT5zIV1Z9sPbdAlKRExr,0.000001,A#,0.3050,-4.451,1,0.3320,183.331,4,Rose Garden,0.835
3,0.73100,Laiss Toi Aime,Tabu Ley Rochereau,0.550,341040,0.341,2i7hxvOTfTjtsEJuWYnKg0,0.871000,A,0.1410,-11.803,1,0.0414,89.147,4,Marie Maria,0.405
4,0.55500,Laiss Toi Aime,Tabu Ley Rochereau,0.696,326760,0.582,0gflenw36L3PVpVUqtRZJd,0.908000,G#,0.1440,-12.348,1,0.0410,129.512,4,Mokrano,0.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.00617,Tala Tannam (Debmaster Remix),Mdou Moctar,0.523,175714,0.675,4W3iduYwzo3hbpvOnN1Otw,0.017100,D,0.3720,-8.344,1,0.0301,84.038,4,Tala Tannam - Debmaster Remix,0.196
205,0.71400,Red Rubber Ball (A Collection),The Cyrkle,0.696,138133,0.133,5dIcC2RTD2BOHTC1F0jrIN,0.000010,F#,0.1100,-20.836,1,0.0387,116.456,4,The Visit (She Was Here),0.653
206,0.77600,Refavela,Gilberto Gil,0.528,238880,0.446,6wPo638YVloLLGDPZRBE2F,0.002530,F#,0.0903,-9.587,1,0.0360,77.518,4,Era nova,0.397
207,0.93300,Fuente Y Caudal,Paco de Lucía,0.458,361867,0.657,3Fgk17TfdjXWFroJy29pXH,0.878000,E,0.0842,-12.020,0,0.0346,102.450,4,Entre Dos Aguas - Instrumental,0.914


In [94]:
merged['mode'] = np.where(merged['mode'] == 1, 'major', 'minor')
# merged_cond_test.loc[merged_cond_test['mode'] == 0, 'key'] = f"{merged_cond_test['key']} minor"

In [95]:
merged['key_signature'] = merged['key'] + ' ' + merged['mode']
merged['key_signature']

0       C major
1       G minor
2      A# major
3       A major
4      G# major
         ...   
204     D major
205    F# major
206    F# major
207     E minor
208    E# major
Name: key_signature, Length: 209, dtype: object

In [96]:
merged.query('time_signature != 4')

,acousticness,album,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,valence,key_signature
12,0.86800,Rivers of Babylon: The Best of The Melodians 1...,The Melodians,0.667,158760,0.3970,50wl4ceK1FASG2rGkkp2fT,0.006150,C#,0.0786,-10.929,major,0.0453,167.825,3,It Comes and Goes,0.8990,C# major
13,0.66800,Liquid Courage,Wax,0.527,241373,0.6570,0SyWTTxLNerKJchqgzw0JW,0.000000,A,0.4910,-6.056,minor,0.2670,78.348,1,Sunday in the City,0.5630,A minor
41,0.92700,Um Violão em Primeiro Plano,Rosinha De Valença,0.445,252413,0.3440,6p5QJBr5iE2xvHatGebTbR,0.738000,F#,0.0999,-11.311,major,0.0347,135.363,5,Asa Branca,0.4290,F# major
57,0.85400,João Gilberto (1973),João Gilberto,0.458,397360,0.0352,2oLrUwf5PeLY2oB6K5O9jN,0.903000,G,0.1160,-27.066,major,0.0510,114.418,3,Undiú,0.2000,G major
58,0.87600,João Gilberto (1973),João Gilberto,0.552,199227,0.0770,5oCezvp8Xm4k6ZgrnPenMc,0.871000,D,0.1110,-23.746,major,0.0300,102.142,3,Valsa (Como Sao Lindos os Youguis),0.3920,D major
61,0.96800,Nara Dez Anos Depois,Nara Leão,0.510,155867,0.1640,6JOUzBzOut78ryNFuCc39k,0.003710,E#,0.1060,-14.807,minor,0.0409,138.362,3,Corcovado,0.2320,E# minor
62,0.92500,Nara Dez Anos Depois,Nara Leão,0.543,122773,0.2430,6UOy5Ne0SGvbozHrLlLJ1J,0.000063,C,0.1210,-13.387,major,0.0517,107.743,5,Chega De Saudade,0.3720,C major
65,0.34900,Estudando o samba,Tom Zé,0.667,198933,0.2060,1brWhQEoBoT9tMnNfcRsjr,0.000390,G#,0.1340,-14.884,minor,0.3250,148.450,3,A felicidade,0.4080,G# minor
70,0.81200,Os Afro-Sambas,Baden Powell,0.265,363267,0.2520,1FM6WoiD8RPTAzozx56wvi,0.132000,C#,0.0712,-18.445,minor,0.0354,96.737,3,Canto de Yemanjá,0.1940,C# minor
88,0.13800,BE MY LOVER,Jizzle,0.609,165619,0.7080,3pW9AlECzn3lmAEMAE9yce,0.000000,D,0.1270,-7.864,major,0.3150,67.121,5,BE MY LOVER,0.6640,D major


In [97]:
# Tempo range, if desired.
tempo_min = 0
tempo_max = 999

# 
friendly_keys = {'C# major': ['C# major', 'C# minor', 'F# major', 'D# minor', 'G# major','A# minor', 'F minor',]}

# Show tracks with harmonically compatible key signatures.
merged.query('key_signature in @friendly_keys["C# major"] & (@tempo_max > tempo > @tempo_min)')


,acousticness,album,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,valence,key_signature
4,0.55500,Laiss Toi Aime,Tabu Ley Rochereau,0.696,326760,0.582,0gflenw36L3PVpVUqtRZJd,0.908000,G#,0.1440,-12.348,major,0.0410,129.512,4,Mokrano,0.946,G# major
7,0.67700,Let Me Down Easy,Derrick Harriott,0.805,312327,0.573,6IrbeEhmzZo0e2JVJLgwxo,0.214000,F#,0.1410,-9.727,major,0.1430,81.747,4,Let Me Down Easy,0.541,F# major
12,0.86800,Rivers of Babylon: The Best of The Melodians 1...,The Melodians,0.667,158760,0.397,50wl4ceK1FASG2rGkkp2fT,0.006150,C#,0.0786,-10.929,major,0.0453,167.825,3,It Comes and Goes,0.899,C# major
15,0.11600,Anthology: Israelites 1963 - 1999,Desmond Dekker & The Aces,0.877,154160,0.556,5uStwsJKG5PvUyeC5XD5vD,0.115000,G#,0.0758,-7.606,major,0.1150,101.686,4,0.0.7 (Shanty Town),0.894,G# major
17,0.14900,Heart Of The Congos (40th Anniversary Edition),The Congos,0.599,365693,0.720,5SBq56S9xKJoH4KyBBr1Oq,0.000003,A#,0.1110,-6.109,minor,0.1280,167.613,4,Fisherman,0.441,A# minor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.00893,African Scream Contest: Raw & Psychedelic Afro...,Lokonon Andre,0.553,234040,0.872,060cPtJougJVQbEMa0bk8D,0.000000,C#,0.0830,-5.758,major,0.1090,185.689,3,Mi Kple Dogbekpo,0.810,C# major
199,0.65600,Nigeria 70 - Funky Lagos,B.Lo,0.385,366840,0.671,0hYEhK3iGxKzsq2zll840N,0.173000,F#,0.3490,-8.753,major,0.0341,139.103,3,Chant To Mother Earth,0.605,F# major
203,0.47700,Soy Lo Que Soy,Monogem,0.488,228440,0.479,2NooozzLR6L6IjLGdCOEqh,0.809000,G#,0.1910,-9.501,major,0.1110,86.973,4,Soy Lo Que Soy,0.576,G# major
205,0.71400,Red Rubber Ball (A Collection),The Cyrkle,0.696,138133,0.133,5dIcC2RTD2BOHTC1F0jrIN,0.000010,F#,0.1100,-20.836,major,0.0387,116.456,4,The Visit (She Was Here),0.653,F# major
